In [ ]:

# Import necessary libraries
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split
import joblib
import dill
from datetime import datetime

In [ ]:
# Load data
df = pd.read_csv('../data/processed/cleaned_data_20240621_143909.csv')

In [ ]:

# Drop duplicates and irrelevant columns
df.drop_duplicates(inplace=True)
df = df.drop(['Unnamed: 0'], axis=1, errors='ignore')

In [ ]:
# Define the numerical and categorical features
numeric_features = ['loan_amount', 'number_of_defaults', 'outstanding_balance', 'interest_rate', 'age', 'remaining_term', 'salary']
categorical_features = ['gender', 'disbursement_date', 'currency', 'country', 'is_employed', 'job', 'location', 'marital_status', 'loan_status']


In [ ]:
# Binning functions
def bin_age(age):
    bins = [0, 18, 30, 40, 50, float('inf')]
    labels = ['0-18', '19-30', '31-40', '41-50', '51+']
    return pd.cut(age, bins=bins, labels=labels, right=False).astype(str)

def bin_salary(salary):
    salary_bins = [0, 2273.93, 2665.44, 3146.58, 10000]
    salary_labels = ['Low', 'Medium-Low', 'Medium-High', 'High']
    return pd.cut(salary, bins=salary_bins, labels=salary_labels, right=False).astype(str)


In [ ]:
# Custom transformer for age binning
def age_bin_transformer(X):
    return pd.DataFrame(X).apply(lambda col: bin_age(col)).values

# Custom transformer for salary binning
def salary_bin_transformer(X):
    return pd.DataFrame(X).apply(lambda col: bin_salary(col)).values


In [ ]:
# Define the preprocessing steps for numerical and categorical features
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

In [ ]:
# Custom transformer for salary binning
def salary_bin_transformer(X):
    return pd.DataFrame(X).apply(lambda col: bin_salary(col)).values

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('age_bin', FunctionTransformer(age_bin_transformer), ['age']),
        ('salary_bin', FunctionTransformer(salary_bin_transformer), ['salary'])
    ],
    remainder='passthrough'
)

In [ ]:
# Define the full pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

In [ ]:
X = df.drop(columns=['target'])
y = df['target']

In [ ]:
X.head()

In [ ]:
# Combine transformations
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features),
        ('age_bin', Pipeline([
            ('binning', FunctionTransformer(age_bin_transformer)),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), ['age']),
        ('salary_bin', Pipeline([
            ('binning', FunctionTransformer(salary_bin_transformer)),
            ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
        ]), ['salary'])
    ],
    remainder='passthrough'
)

In [ ]:
# Define the full pipeline
pipeline = Pipeline(steps=[('preprocessor', preprocessor)])

# Separate features and target
X = df.drop(columns=['target'])
y = df['target']

In [33]:
# Fit the pipeline and transform the data
pipeline.fit(X)
X_transformed = pipeline.transform(X)


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [34]:
# Save the preprocessed data and the pipeline
pd.DataFrame(X_train).to_csv('../data/train/X_train.csv', index=False)
pd.DataFrame(X_test).to_csv('../data/test/X_test.csv', index=False)
pd.DataFrame(y_train).to_csv('../data/train/y_train.csv', index=False)
pd.DataFrame(y_test).to_csv('../data/test/y_test.csv', index=False)

In [35]:
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
with open(f'../pipelines/data_processing_pipeline_{timestamp}.pkl', 'wb') as f:
    dill.dump(pipeline, f)